In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = tf.cast(x_train, tf.float32), tf.cast(x_test, tf.float32)

num_classes = 10
num_features = x_train.shape[1] * x_train.shape[2]

x_train, x_test = tf.reshape(x_train, [-1, num_features]), tf.reshape(x_test, [-1, num_features])

x_train, x_test = x_train / 255., x_test / 255.

In [ ]:
print(x_train.dtype)

In [ ]:
print(x_test.shape)  # 28*28
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(x_train[2])

In [ ]:
from tensorflow.keras import Model, layers

class NeuralNet(Model):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = layers.Dense(128, activation=tf.nn.relu)
        self.fc2 = layers.Dense(256, activation=tf.nn.relu)
        self.fc3 = layers.Dense(num_classes)

    def call(self, x, is_training=False):
        o1 = self.fc1(x)
        o2 = self.fc2(o1)  #60000,128 -> 60000, 256
        o3 = self.fc3(o2)  #60000,10
        return o3

        if not is_training:
            x = tf.nn.softmax(x)
        return x

neural_net = NeuralNet()

In [ ]:
def cross_entropy_loss(x, y): # with_logits --> logits를 쓰는게 numerical stable하다
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # y [60000,1]
    # x [60000,10]
    # loss [60000, 1] -> [1,1]
    return tf.reduce_mean(loss, axis=0)

In [ ]:
# Accuracy metric.
def accuracy(y_pred, y_true):
    # y_pred [60000,10]
    # y_true [60000,1]
    correct_prediction = tf.equal(tf.argmax(y_pred, axis=1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
lr = 0.003
optimizer = tf.optimizers.SGD(lr)

def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy_loss(pred, y)

    trainable_variables = neural_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
# Use tf.data API to shuffle and batch data.
batch_size = 200
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size).prefetch(1)

epoch = 40
display_epoch = 4

# Run training for the given number of steps.
for epo in range(1,epoch+1):
    for step, (batch_x, batch_y) in enumerate(train_data, 1):
        run_optimization(batch_x, batch_y)

        if epo & display_epoch == 0:
            pred = neural_net(batch_x)
            loss = cross_entropy_loss(pred, batch_y)
            acc = accuracy(pred, batch_y)
            print("epoch: %i, step: %i, loss: %f, accuracy: %f" % (epo, step, loss, acc))

In [ ]:
pred = neural_net(x_test)
print("Test Accuracy", accuracy(pred, y_test).numpy())

In [ ]:
# Predict 5 images from validation set.
n_images = 5
test_images = x_test[:n_images]
prediction = neural_net(test_images)

# Display image and model prediction.
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction: %i" % np.argmax(prediction.numpy()[i]))

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(10)
])

In [ ]:
model2.compile(optimizer="sgd", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

In [ ]:
inform = model2.fit(x_train, y_train, epochs=40, batch_size=128, validation_data=(x_test, y_test))
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(inform.history['loss'], 'y', label='train loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.plot(inform.history['accuracy'], 'b', label='train acc')